# Buchhandelsdaten in Vufind als Grundlage für PDA (Patron Driven Aquisition) am MPIfG / Version 2: Updates laden statt Vollimport

Einbindung von freundlicherweise von Schweitzer Fachinformation zur Verfügung gestellten Daten.   
(Auswahl über passend konfigurierte Neuerscheinungsabfragen in unserem Kundenprofil).   

#### Gründe für die Umstellung auf Updates: Durch Einspielen der Daten in Vufind ergibt sich eine zeitliche Diskrepanz, in der Titel bereits in Aleph gelöscht sind, aber in Vufind noch bestellbar. Zudem werden die Aleph-IDs hochgezähl, denn bei jedem Einspielen werden neue Nummern vergeben. Durch die Updates bleiben die Titel weiterhin verfügbar und das Hochzählen der IDs hält sich im Rahmen. Das Skript beschleunigt sich durch Reduzierung der Abfragen am Server.
*Kleiner Nachteil: der manuelle Aufwand vergrößert sich. Mehrere Dateien müssen in Aleph eingespielt und verarbeitet werden, aber hält sich nach wie vor im vertretbaren Rahmen.*

Das Jupyter Notebook arbeitet mit Python 3.9.1 und wurde mit Visual Studio Code 1.62.3 erstellt 


#### Arbeitsschritte im Code:

> Vorarbeiten:   
  - Notwendige Pandas Libraries aufrufen
  - Serverprüfung auf funktionierende Verbindung zum Aleph-X-Server    


> Daten abholen und einlesen:   
  1. Buchhandelsdaten von Schweitzer 
  2. Aleph-Konkordanz Aleph-ID /Schweitzer ID  (erzeugt tagesaktuell per p-print-03 in Aleph)
     - Aufbereiten der Daten: Schweitzer ID extrahieren und Aleph-ID mit Nullen auffüllen
  3. Daten zusammenführen in einem df    


> Daten aufbereiten:   
  1. Buchhandelsdaten prüfen und vorbereiten
     1. Identifizierung von Titeln in Aleph, die nicht mehr im Datensatz sind und schreiben in Datei "ges02_loeschen_1"
     2. Dublettencheck innerhalb der Buchhandelsdaten
     3. Trennung der Daten in "in Aleph vorhanden" und "neu"  
  2. Bestandsabfragen:
     1. Ganz neue Titel   
        - Bestandsabgleich durch Abfragen (GES und EBX) auf dem Aleph-Server   
     2. In Aleph vorhanden
        - Bestandsabgleich durch Abfrage EBX auf dem Aleph-Server  
     3. Exportvorbereitungen:
        1. Neue Titel
        2. In Aleph vorhandene Titel   
            Identifizierung von zu löschenden Titeln "ges02_loeschen_2" und der zu aktualisierenden
  3. Exportdateien aufbereiten:
     1. Aufbereiten neuer Titel
     2. Aufbereiten der vorhandenen Titel   


> Informationssammlung
   1. Log-Datei mit Rahmendaten wird fortlaufend geschrieben
   2. Ausgabe bestimmter Titelgruppen als csv-Datei  


<hr>

## Vorarbeiten

### Pandas Libraries laden

In [1]:
import pandas as pd                                                # für das Arbeiten mit der CSV-Datei
import urllib.request                                              # für das Abrufen der URL
import requests                                                    #für die Bestandsabfragen 
pd.options.mode.chained_assignment = None                          # default='warn' abschalten beim Beschreiben der neuen Spalten
import time                                                        # für das Schreiben des Datums Logdatei und Excel-Export und Arbeiten mit dem Erscheinungsdatum
import datetime                                                    # für das Berechnen des Updates
import numpy as np                                                 # für das Bearbeiten von Spalten

### Prüfung, ob die Verbindung zum Aleph-Server für Abfragen korrekt funktioniert:

    Nur zugelassene IPs können diese Schnittstelle abfragen.

In [86]:
test= "http://aleph.mpg.de/X?op=find&base=ges01&request=IBS=9783482648434"

reply = requests.get(test).text
a = reply.find("Forbidden")
b =  reply.find("?xml")

if (a > 50):
    print("Es gibt ein Problem mit dem Server")
if (b == 1):
    print("Der Server antwortet korrekt")

Der Server antwortet korrekt


<hr>


## Datensätze abholen und einlesen

### 1. Datensätze von Schweitzer

*einlesen in df*

In [2]:
url = "https://content.schweitzer-online.de/static/content/export/mpifg/export.csv"  # Abruf, der von Schweitzer zur Verfügung gestellten Daten
checkout_file = "./input/export.csv"  
urllib.request.urlretrieve(url, checkout_file)

('./input/export.csv', <http.client.HTTPMessage at 0x1befed343d0>)

In [59]:
df = pd.read_csv('./input/export.csv', encoding = 'UTF-8', sep=';' , keep_default_na=False) # muss encoding angeben und Trennzeichen, NaN (= leere Werte) direkt beim Import entfernen, da sie später Probleme machen

In [60]:
#df

,object_id,isbn_ean,title,subtitle,contributor,publisher,series,thesis,edition_number,edition_text,...,publication_year,publication_date,delivery_signal,delivery_text,price,description,cover,predecessor,follower,last_modified
0,160009340,9783593448084,Kalkulierte Mobilität,Ökonomische und biographische Perspektiven auf...,Judith Schmidt,Campus,Arbeit und Alltag 20,,1,,...,2021,20210721,green,delivery_status_econtent:available,3699,Saisonarbeit ist eine bedeutende Variable der ...,https://content.schweitzer-online.de/static/ca...,,,2021-11-18 18:24:57
1,159663468,9783161564154,Die Tradition staatlicher Interventionen in de...,,Valesca Maria Molinari,Mohr Siebeck,Rechtsordnung und Wirtschaftsgeschichte,Dissertationsschrift,1,,...,2021,20211100,green,delivery_status_econtent:available,5900,Die Entwicklungen auf dem Mietwohnungsmarkt de...,https://content.schweitzer-online.de/static/ca...,,,2021-11-12 18:15:00
2,158945668,9780231550734,Experiencing Design,The Innovator's Journey,Jeanne Liedtka;Karen Hold;Jessica Eldridge,Columbia University Press,Columbia Business School Publishing,,1,,...,2021,20210908,green,delivery_status_econtent:available,2799,Individuals become design thinkers by experien...,https://content.schweitzer-online.de/static/ca...,,,2021-11-24 17:20:02
3,161161334,9783161608469,Im Olymp der Ökonomen,Zur öffentlichen Resonanz wirtschaftspolitisch...,Lino Wehrheim,Mohr Siebeck,Die Einheit der Gesellschaftswissenschaften im...,Dissertationsschrift,1,1. Auflage 2021,...,2021,20210916,green,delivery_status_econtent:available,9900,In Zeiten von Fake News und alternativen Wahrh...,https://content.schweitzer-online.de/static/ca...,,,2021-11-28 20:56:06
4,161502056,9783496030478,Die Südseeblase in der englischen Kunst des 18...,Bilder einer Finanzkrise,Kathleen Löwe,"Reimer, Dietrich",,,1,,...,2021,20210709,green,delivery_status_econtent:available,4900,"Als es 1720 in England zu einem Börsenkrach, d...",https://content.schweitzer-online.de/static/ca...,,,2021-11-19 18:27:38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7045,161277585,9781800713031,The Creative Industries and International Busi...,,Nnamdi O. Madichie;Robert Ebo Hinson,Emerald Publishing Limited,,,,,...,2022,20220121,yellow,delivery_status:10,8287,The international business environment has und...,https://content.schweitzer-online.de/static/ca...,,,2021-11-19 18:09:58
7046,161277677,9781801171809,Unravelling Travelling,Uncovering Tourist Emotions through Autoethnog...,Sue Beeton,Emerald Publishing Limited,The Tourist Experience,,,,...,2022,20220120,yellow,delivery_status:10,8924,"The Tourist Experience is complex, intrinsical...",https://content.schweitzer-online.de/static/ca...,,,2021-11-19 18:10:25
7047,162078229,9780745340173,Work and the Carceral State,,Jon Burnett,Pluto Press,,,,,...,2022,20220120,yellow,delivery_status:10,9484,'Revolutionises our understanding of the carce...,,,,2021-11-29 17:22:03
7048,162745003,9781119745976,The Strategyzer Box Set,,Alexander Osterwalder,Wiley,The Strategyzer series,,,,...,2020,20201203,green,delivery_period:available_15to20days,17078,The Authoritative Library for Business Leaders...,https://content.schweitzer-online.de/static/ca...,,,2021-11-29 17:35:23


### 2. Aleph-Konkordanz Aleph-ID / Schweitzer ID

In [61]:
now = time.strftime('%Y%m%d')  # %H:%M:%S
input_file = './input/ids'+now      #input_file wird tagesaktuell aus Aleph gezogen und auf diesem Wege mit der entsprechenden Endung eingelesen

In [62]:

df_alephIDs = pd.read_fwf(input_file, encoding = 'UTF-8', sep='\s\s', header=None, keep_default_na=False) 
df_alephIDs.columns=["ID","Field","L","Content"]
df_alephIDs

#Spalte 0 = Aleph-IDs müsste vor Export mit Nullen aufgefüllt werden 
#Spalte 3 = object_id aus df, hier muss zum Abgleich $$a entfernt werden


,ID,Field,L,Content
0,62432,20,L,$$a160009340
1,62433,20,L,$$a158945668
2,62434,20,L,$$a161502056
3,62435,20,L,$$a159450684
4,62436,20,L,$$a156899389
...,...,...,...,...
6195,68627,20,L,$$a161219987
6196,68628,20,L,$$a160093944
6197,68629,20,L,$$a2543785
6198,68630,20,L,$$a159457857


*Datenfelder aufbereiten: Schweitzer-ID extrahieren und Aleph-ID ins richtige Format bringen*

In [63]:
df_alephIDs["object_id"] = df_alephIDs["Content"].astype(str).str.slice(start=3,stop=13).apply(int)   #Spalte mit object-Ids herausschneiden und wieder zur Zahl definieren

df_alephIDs["object_id"]

0       160009340
1       158945668
2       161502056
3       159450684
4       156899389
          ...    
6195    161219987
6196    160093944
6197      2543785
6198    159457857
6199    161718808
Name: object_id, Length: 6200, dtype: int64

In [64]:
#Aleph-ID-Spalte mit Nullen Auffüllen
df_alephIDs["ids"] = df_alephIDs['ID'].apply(lambda x: f"{x:09d}")
df_alephIDs["ids"]

0       000062432
1       000062433
2       000062434
3       000062435
4       000062436
          ...    
6195    000068627
6196    000068628
6197    000068629
6198    000068630
6199    000068631
Name: ids, Length: 6200, dtype: object

### 3. Daten zu einem Frame zusammenführen

In [11]:
df_oi= df.set_index("object_id")                       #object-ID zum Index für beide Datenframes
df_aleph_oi = df_alephIDs.set_index("object_id")
df_update = df_oi.join(df_aleph_oi)                    #dann können sie zusammengeführt werden

In [70]:
#df_update

,isbn_ean,title,subtitle,contributor,publisher,series,thesis,edition_number,edition_text,media_type,...,description,cover,predecessor,follower,last_modified,ID,Field,L,Content,ids
object_id,,,,,,,,,,,,,,,,,,,,,
160009340,9783593448084,Kalkulierte Mobilität,Ökonomische und biographische Perspektiven auf...,Judith Schmidt,Campus,Arbeit und Alltag 20,,1,,ebook,...,Saisonarbeit ist eine bedeutende Variable der ...,https://content.schweitzer-online.de/static/ca...,,,2021-11-18 18:24:57,62432.0,20.0,L,$$a160009340,000062432
159663468,9783161564154,Die Tradition staatlicher Interventionen in de...,,Valesca Maria Molinari,Mohr Siebeck,Rechtsordnung und Wirtschaftsgeschichte,Dissertationsschrift,1,,ebook,...,Die Entwicklungen auf dem Mietwohnungsmarkt de...,https://content.schweitzer-online.de/static/ca...,,,2021-11-12 18:15:00,68009.0,20.0,L,$$a159663468,000068009
158945668,9780231550734,Experiencing Design,The Innovator's Journey,Jeanne Liedtka;Karen Hold;Jessica Eldridge,Columbia University Press,Columbia Business School Publishing,,1,,ebook,...,Individuals become design thinkers by experien...,https://content.schweitzer-online.de/static/ca...,,,2021-11-24 17:20:02,62433.0,20.0,L,$$a158945668,000062433
161161334,9783161608469,Im Olymp der Ökonomen,Zur öffentlichen Resonanz wirtschaftspolitisch...,Lino Wehrheim,Mohr Siebeck,Die Einheit der Gesellschaftswissenschaften im...,Dissertationsschrift,1,1. Auflage 2021,ebook,...,In Zeiten von Fake News und alternativen Wahrh...,https://content.schweitzer-online.de/static/ca...,,,2021-11-28 20:56:06,67902.0,20.0,L,$$a161161334,000067902
161502056,9783496030478,Die Südseeblase in der englischen Kunst des 18...,Bilder einer Finanzkrise,Kathleen Löwe,"Reimer, Dietrich",,,1,,ebook,...,"Als es 1720 in England zu einem Börsenkrach, d...",https://content.schweitzer-online.de/static/ca...,,,2021-11-19 18:27:38,62434.0,20.0,L,$$a161502056,000062434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161277585,9781800713031,The Creative Industries and International Busi...,,Nnamdi O. Madichie;Robert Ebo Hinson,Emerald Publishing Limited,,,,,hardcover,...,The international business environment has und...,https://content.schweitzer-online.de/static/ca...,,,2021-11-19 18:09:58,NaN,NaN,NaN,NaN,NaN
161277677,9781801171809,Unravelling Travelling,Uncovering Tourist Emotions through Autoethnog...,Sue Beeton,Emerald Publishing Limited,The Tourist Experience,,,,hardcover,...,"The Tourist Experience is complex, intrinsical...",https://content.schweitzer-online.de/static/ca...,,,2021-11-19 18:10:25,NaN,NaN,NaN,NaN,NaN
162078229,9780745340173,Work and the Carceral State,,Jon Burnett,Pluto Press,,,,,hardcover,...,'Revolutionises our understanding of the carce...,,,,2021-11-29 17:22:03,NaN,NaN,NaN,NaN,NaN


<hr>

### LOG-Datei für den Prozess, zur Dokumentation des Imports und als Kontrollanzeige hier


In [72]:
x = df_update.shape[0]
print('Anzahl der enthaltenen Datensätze:', x)
#print('vorhandene ISBNs:', df_update["isbn_ean"].shape[0])

#z = df_alephIDs["ids"].count()
z= df_aleph_vorhanden["ids"].count()
y = df_update["ids"].count()

print('-------------------------')
print('Aleph-IDs anfangs', z)
print('Aleph-IDs nach join' , y)

timestr = time.strftime('%d.%m.%Y - %H:%M')

with open ('./log/pda_import_log.txt', 'a') as log:                                                  # Da diese Log-Datei nicht unmittelbar gebraucht wird, hier fortlaufendes Schreiben in eine Datei
    log.write('Logdatei PDA-Import vom ')
    log.write(timestr)
    log.write('\n------------------------------------------\n')
    log.write('Gelieferte Datensätze:             ' + str(x))
    log.write('Datensätze in Aleph:               ' + str(z))

Anzahl der enthaltenen Datensätze: 7050
-------------------------
Aleph-IDs anfangs 5974
Aleph-IDs nach join 5970


<hr>

# 4 IDs verschwinden, sind dann nicht mehr vorhanden, die muss ich noch identifizieren!

<hr>

# Problemlösung der Aleph-Dubletten
## nur in Aleph aber nicht mehr im Import, diese müssen identifiziert werden und ggf. gelöscht!

In [65]:
df_alephIDs["object_id"].value_counts

df_aleph_einzel = df_alephIDs.drop_duplicates("object_id", keep=False)  #Auslesen der Eintraege mit einzelner object_id

In [66]:
df_aleph_doppelt = df_alephIDs.groupby("object_id").filter(lambda g: (g.nunique() >1).any()) #Doppelte object_ids rausziehen
df_aleph_single = df_aleph_doppelt.sort_values(by=["object_id", "ids"], ascending =False).drop_duplicates(subset=["object_id"], keep='first')  #von Dubletten 1. behalten 

In [67]:
df_aleph_vorhanden = df_aleph_einzel.append(df_aleph_single)

In [68]:
df_oi= df.set_index("object_id")                       #object-ID zum Index für beide Datenframes
df_aleph_oi = df_aleph_vorhanden.set_index("object_id")
df_update = df_oi.join(df_aleph_oi)                    #dann können sie zusammengeführt werden 


<hr>

# Nächsten Schritt noch machen, um Daten zu löschen!!

<hr>

## Daten aufbereiten

<hr>

### 1. Identifizierung von Titeln in Aleph, die nicht mehr im Datensatz sind

In [69]:
# andere Kriterien!!! Alle, bei denen Titel leer ist z.b. oder geht das nicht, weil die gar nicht gemappt werden können bei Join???

df_aleph_loeschen = df_aleph_doppelt.sort_values(by=["object_id", "ids"], ascending =False).drop_duplicates(subset=["object_id"], keep='last') #Datei der zu löschenden, Aleph-IDs noch rausschreiben und mit GES02 anreichern, dann kann es Eingabedatei zum Löschen sein

In [94]:
with open("./output/ges02_loeschen_1", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_aleph_loeschen.index:
        fa.write(df_aleph_loeschen["ids"][i]+'GES02'+'\n')

In [73]:
df_update.reset_index(inplace=True) 

<hr>

## 2. Dublettencheck innerhalb der Buchhandelsdaten 

Aufgaben im Rahmen des Dublettencheck:
1. Dublettenkontrolle anhand von Titel, Untertitel und Autor 
   - Zunächst Behebung der unsauberen Titel / Untertitel-Trennung für korrekteren Abgleich
   - Trennung der Datensätze in Dubletten und "Einzeltitel"
     - Einzeltitel werden direkt für Bestandsprüfung vorgemerkt
     - Dubletten werden auf neueste Version reduziert und diese der Bestandsprüfungsdatei hinzugefügt

### Die Dublettenkontrolle Anhand von Titel, Untertitel und Autor

*Entfernen von Untertiteln aus der Titelspalte, Extrahieren von Untertiteln und Abgleich mit Untertitelspalte und Schreiben der vorhandenen Informationen in neue Untertitel-Spalte.   
Durch diese Spalte werden ca. 1/3 mehr Dubletten erkannt, als ohne die Bereinigung. *

In [76]:
#Update-Datei wieder umbenennung zu df, um Skript nicht komplett umschreiben zu  müssen
df = df_update

In [77]:
neu = df["title"].str.split(':', n = 1, expand = True)  #Titel am 1. Doppelpunkt splitten und getrennt in neue Felder schreiben
df["title_sep"]= neu[0]
df["subtitle_sep"]= neu[1]

df["subtitle_sep"] = df["subtitle_sep"].replace(np.nan, '', regex=True) #NaN-Werte stören, darum raus damit ...

In [78]:
comparison = np.where(df["subtitle"] == df["subtitle_sep"], '', df["subtitle"])    # Abgleich - wenn in beiden das Gleiche steht, dann ursprüngliches "Subtitle"-Feld nehmen
df["subtitle_comparison"] = comparison  

comparison2 = np.where(df["subtitle"] < df["subtitle_sep"], df["subtitle_sep"], '') # Wenn nur in "subtitle_sep" Infos stehen, diese übernehmen, das ist noch nicht ganz sauber, da hier manchmal anderes steht als in "subtitle"
df["subtitle_comparison2"] = comparison2 

df["subtitle_all"] = df["subtitle_comparison"]+df["subtitle_comparison2"]          # Beide Informationen in neuer Subtitle-Spalte zusammenführen

In [79]:
df["short_title"] = df["title_sep"] + ' ' + df["subtitle_all"] + ' / ' + df["contributor"]  # aus den bereinigten Daten einen Kurztitel erzeugen, der dann für den Dublettencheck verwendet wird

In [80]:
df_dubletten = df.groupby("short_title").filter(lambda g: (g.nunique() >1).any()) # schreibt alle mehrfach vorhandenen Titel in ein eigenes Datenframe

In [81]:

df_dubl_einspielen = df_dubletten.sort_values(by=["short_title", "publication_year"], ascending =False).drop_duplicates(subset=["short_title"], keep='first')   # sortiert Dubletten nach Jahr und schreibt den jeweils ersten (= neuesten) Eintrag in neues Dataframe


In [82]:
df_ohne_dubletten = df.drop_duplicates("short_title", keep=False)       #durch ""keep=False" werden alle nicht-Dubletten rausgezogen

In [83]:
df_dubl_nicht_einspielen = df_dubletten.sort_values(by=["short_title", "publication_year"], ascending =False).drop_duplicates(subset=["short_title"], keep=False)

#Kontrolle, ob es in Dubletten Titel gibt, die schon in Aleph sind
df_dubl_nicht_einspielen["ids"].count()


In [85]:
df_einspielen = df_ohne_dubletten.append(df_dubl_einspielen)                    # die ausgewählten Dubletten und alle Nicht-Dubletten werden in ein Datenframe zusammengeführt
df_einspielen.reset_index(inplace=True)                                         # für weitere Bearbeitung index-reset nötig

<hr>

## 3. Überblick zu den Daten und Trennen in "in Aleph vorhanden" und "neu"

In [86]:
m = df_einspielen.shape[0]
n = df_einspielen["ids"].count()
print("Zum Einspielen:", m)
print("Davon in Aleph:", n)
print("Neue Titel:", m-n)

Zum Einspielen: 6217
Davon in Aleph: 5740
Neue Titel: 477


In [87]:
df_in_aleph = df_einspielen.dropna(subset=['ids'])
df_in_aleph.reset_index(inplace=True)
df_in_aleph.shape[0]

# diese müssen nur am Ebx-Bestand geprüft werden

5740

In [88]:
df_ganz_neu = df_einspielen[df_einspielen['ids'].isnull()]

df_ganz_neu.reset_index(inplace=True)
df_ganz_neu.shape[0]

# diese müssen am Ges und Ebx Bestand geprüft werden

477

<hr>

## 2. Bestandsabfragen



*URLs für die Abfrage über den X-Server unseres Bibliothekssystems werden erzeugt und über die ISBN eine Abfrage auf Bestand gemacht. Die Abfrage funktioniert nur für zugelassene IPs (darum oben die Prüfung).  
Für die Abfrage in unseren Bestand ist die ISBN sehr gut, da in den Titeldaten alle im Buch befindlichen ISBNs - auch die anderer Ausgabeformen - mit übernommen sind. Beim MPG-Ebooks Katalog handelt sich um Daten von Verlagen, die sich in ihrer Qualität und Informationsumfang sehr unterscheiden. Hier wird noch zu prüfen sein, inwieweit ein anderer Abfragemechansimus gewählt werden muss.*  

### 1. Ganz neue Titel  

= df_ganz_neu


In [89]:
df_ganz_neu["url_ges"] = df_ganz_neu["isbn_ean"].apply(lambda x: f"ges_link{x}".replace('ges_link','http://aleph.mpg.de/X?op=find&base=ges01&request=IBS='))

#Es funktionierte nicht, dass die URLs an die vorhandenen ISBNs einfach so angefügt werden, darum der Workaround mit einem Platzhalter, der sich dann über replace vom richtigen Link überschreiben ließ.

#### URLs für Ebooks-Katalog erzeugen 

In [90]:
df_ganz_neu["url_ebx"] = df_ganz_neu["isbn_ean"].apply(lambda x: f"ebx_link{x}".replace('ebx_link', 'http://aleph.mpg.de/X?op=find&base=ebx01&request=IBN='))


#### Abfragen beim Server

##### Zunächst für die Daten des MPIfG 

*Vorgehensweise: Abfrage und Sammeln der Antworten in einer Datei, diese Antworten werden dann in Ausdrücke "übersetzt" - "vorhanden" und "neu" und diese Daten in eine Spalte ins Dataframe zur weiteren Auswertung übertragen.   
Schwierigkeit hier war, die Sammlung der Antworten zu den einzelnen Titeln, um sie in das Datenframe einzuspielen. Der störende XML-Header der Antworten wird erst gar nicht in die Datei geschrieben.   *   

In [91]:
with open('./input/server_responses', 'w') as fn:  
    for url in df_ganz_neu["url_ges"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses', 'r') as f:
    with open('./input/server_responses_transfered', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf = pd.read_fwf('./input/server_responses_transfered', names=["Abfrage_ges"])
df_result = pd.concat([df_ganz_neu, df_fwf], axis=1)             

    Randnotiz: 
    Bei 2400 Titels brauchte der Abgleich ca 350 Sekunden

In [95]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind, wird mittelfristig rausfallen
x = df_ganz_neu.shape[0]
y = df_fwf.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

Anzahl der eingelesenen Datensätze: 477 
Anzahl der Antworten vom Server:    477


##### Datenabgleich mit dem Bestand des MPG Ebooks-Katalog

*Vorgehensweise analog Bestandsabfrage MPI.*

In [96]:
with open('./input/server_responses_ebx', 'w') as fn:  
    for url in df_result["url_ebx"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses_ebx', 'r') as f:
    with open('./input/server_responses_transfered_ebx', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf_ebx = pd.read_fwf('./input/server_responses_transfered_ebx', names=["Abfrage_ebx"])
df_result_neu = pd.concat([df_result, df_fwf_ebx], axis=1)

In [97]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind
x = df_result_neu.shape[0]
y = df_fwf_ebx.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

Anzahl der eingelesenen Datensätze: 477 
Anzahl der Antworten vom Server:    477


<hr>

### 2. Titel in Aleph vorhanden

= df_in_aleph

*Hier reicht Abgleich mit Ebooks, das erworbene Bücher manuell gelöscht werden.*

In [98]:
df_in_aleph["url_ebx"] = df_in_aleph["isbn_ean"].apply(lambda x: f"ebx_link{x}".replace('ebx_link', 'http://aleph.mpg.de/X?op=find&base=ebx01&request=IBN='))

#### Abfragen beim Server

##### bei diesen Daten nur bei den Ebooks

In [99]:
with open('./input/server_responses_ebx2', 'w') as fn:  
    for url in df_in_aleph["url_ebx"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses_ebx2', 'r') as f:
    with open('./input/server_responses_transfered_ebx2', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf_ebx2 = pd.read_fwf('./input/server_responses_transfered_ebx2', names=["Abfrage_ebx"])
df_result_in_aleph = pd.concat([df_in_aleph, df_fwf_ebx2], axis=1)

In [101]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind
x = df_result_in_aleph.shape[0]
y = df_fwf_ebx2.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

Anzahl der eingelesenen Datensätze: 5740 
Anzahl der Antworten vom Server:    5740


<hr>

## 3. Exportvorbereitungen 


### 1. Neue Titel

In [102]:
df_neu_nicht_einspielen = df_result_neu.drop(df_result_neu[(df_result_neu["Abfrage_ebx"]== 'neu') & (df_result_neu["Abfrage_ges"] == 'neu')].index) 
# alle Titel rausholen, die in einer der beiden Datenbanken vorhanden waren, diese werden mit den vorhandenen aus Update unten in Excel geschrieben

#### Jetzt Extraktion der Titel zum Einspielen:

In [103]:
df_aleph_neu_einspielen = df_result_neu.loc[((df_result_neu["Abfrage_ebx"]== 'neu') & (df_result_neu["Abfrage_ges"] == 'neu'))]

### 2. In Aleph vorhandene Titel

*Hier ist der Fall: was im Ebooks-Katalog vorhanden ist, muss gelöscht werden*

In [104]:
df_in_aleph_nicht_einspielen = df_result_in_aleph.drop(df_result_in_aleph[(df_result_in_aleph["Abfrage_ebx"]== 'neu')].index) #alles, was schon im Ebook-Katalog ist, diese Titel müssen gelöscht werden
df_in_aleph_nicht_einspielen

In [180]:
# Erzeugen der Datei zum Löschen der Titel

with open("./output/ges02_loeschen_2", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_in_aleph_nicht_einspielen.index:
        fa.write(df_in_aleph_nicht_einspielen["ids"][i]+'GES02'+'\n')

Für die Statistik und Kontrolle zusammenführen aller Titel, die nicht eingespielt werden und Ausgabe in einer CSV

In [105]:
df_gesamt_nicht_einspielen = df_in_aleph_nicht_einspielen.append(df_neu_nicht_einspielen)   

In [106]:
date = time.strftime("%Y_%m_%d")                                              # Zeit erfassen für Dateibenennung

df_gesamt_nicht_einspielen["object_id"] = df_gesamt_nicht_einspielen.object_id.astype(str)  # wandelt die spalte von Int64 zu Object um, so dass es in Excel korrekt eingelesen wird
df_gesamt_nicht_einspielen["isbn_ean"] = df_gesamt_nicht_einspielen.isbn_ean.astype(str)
df_gesamt_nicht_einspielen = df_gesamt_nicht_einspielen.drop(columns=["url_ebx", "url_ges", "cover", "title_sep", "subtitle_comparison", "subtitle_comparison2", "subtitle_all", "subtitle_sep"]) # unnötige Spalten entfernen

df_neu_nicht_einspielen.to_csv('./output/Vorhandene_Titel_'+date+'.csv')   

ModuleNotFoundError: No module named 'xlsxwriter'

#### Jetzt Extraktion der Titel zum Updaten:

In [108]:
df_in_aleph_update = df_result_in_aleph.loc[(df_result_in_aleph["Abfrage_ebx"]== 'neu')] # prüfen, ob es wirklich ein Datenupdate gab, sonst nicht neu einspielen??

<hr>

#### Für die Logdatei Ermittlung verschiedener Zahlen und hier zur direkten Ansicht ausgegeben

In [111]:
#Kontrollmechanismus, ob für alle Titel auch Treffer da sind
x = df_update.shape[0]  #Ursprungszahl, eigentlich oben schon berechnet ...
a = df_ohne_dubletten.shape[0]
c = df_dubletten.shape[0]
b = df_dubl_einspielen.shape[0] #Auswahl der neuen Treffer
#von oben:
m = df_einspielen.shape[0]
n = df_einspielen["ids"].count()
o = df_in_aleph_update.shape[0]

y = df_gesamt_nicht_einspielen.shape[0] #vorhandene von ganz neuen
z = df_aleph_neu_einspielen.shape[0]       #neu ermitteln aus neuen
d = a + b
print('Kleine Statistik:\n=====================================',
    '\nGelieferte Datensätze:           ', x, 
    '\n-------------------------------------',
    '\nSätze ohne Dubletten:            ', a,
    '\n    Dubletten:       ', c, 
    '\n    Davon zum Einspielen:        ', b,
    '\nAm Bestand abzugleichen:         ', m,
    '\n----------------------------------------------------------------',
    '\nTitel, die schon in Aleph sind:                           ', n,
    '\n    Davon zum updaten:                                    ', o,
    '\n    Davon aus Aleph zu löschen:                           ', n-o,
    '\nTitel, die komplett neu sind:                             ', m-n,
    '\n    Davon zum Einspielen:                                 ', z,
    '\n----------------------------------------------------------------',    
    '\nVorhandene Titel, die nicht in Aleph exportiert werden:   ', y,
    '\n----------------------------------------------------------------',)

#hier entsprechende Einträge für die Log-Datei

with open ('./log/pda_import_log.txt', 'a') as log:
    log.write("\nSätze ohne Dubletten               " + str(a))
    log.write("\n   Dubletten:              " + str(c))
    log.write("\n   Auswahl zum Einspielen:         " + str(b))
    log.write("\n--------------------------------------------\n")
    log.write("\nDavon bereits in Aleph vorhanden:  " + str(n))
    log.write("\nDavon komplett neu in Aleph:       " + str(m-n))   
    log.write("\n--------------------------------------------\n")
    log.write("\nTitel, die eingespielt werden:     " + str(o+z))
    log.write("\nDavon als Update:                  " + str(o))
    log.write("\nDavon ganz neu:                    " + str(z))
    log.write("\n--------------------------------------------\n")
    log.write("\nTitel in Aleph zu löschen:         " + str(n-o)) 
    log.write("\nNicht eingespielt werden:          " + str(y))

Kleine Statistik:
Gelieferte Datensätze:            7050 
------------------------------------- 
Sätze ohne Dubletten:             5428 
    Dubletten:        1622 
    Davon zum Einspielen:         789 
Am Bestand abzugleichen:          6217 
---------------------------------------------------------------- 
Titel, die schon in Aleph sind:                            5740 
    Davon zum updaten:                                     5735 
    Davon aus Aleph zu löschen:                            5 
Titel, die komplett neu sind:                              477 
    Davon zum Einspielen:                                  267 
---------------------------------------------------------------- 
Vorhandene Titel, die nicht in Aleph exportiert werden:    215 
----------------------------------------------------------------


<hr>

## 4. Exportdateien Aufbereiten

#### Zielformat für das Einspielen in Aleph:

    000000001 LDR   L -----nM2.01200024------h              
    000000001 020   L $$a (object_id))
    000000001 030   L $$aaz||rrrza||||
    000000001 051   L $$am|||||||
    000000001 070   L $$aSchweitzer
    000000001 077   L $$aMonographie
    000000001 078   L $$aSchweitzer
    000000001 082   L $$azum Bestellen
    000000001 100   L $$a (contributor_1)
    000000001 104   L $$a (contributor_2)
    000000001 108   L $$a (contributor_3)
    000000001 331   L $$a (title_sep)
    000000001 335   L $$a (subtitle_all)
    000000001 403   L $$a (edition_number / edition_text)  #noch prüfen, was besser zu verwenden ist 
    000000001 419   L $$b (publisher) $$a (date_combined)
    000000001 433   L $$a (pages)
    000000001 451   L $$a (series)
    000000001 520   L $$a (thesis)
    000000001 540   L $$a (isbn_ean)
    000000001 656   L $$a (cover)
    000000001 750   L $$a (description)
    000000001 655   L $$zOrder me$$umailto:bib@mpifg.de?subject=Bestellwunsch        
    
Anmerkung zum Feld 655: die URL wird NACH dem Einspielen in Aleph mit der Datensatz-ID angereichert (siehe Juypter-Notebook "Link-Anreicherung"), um einen klaren Bestellink für den Kaufvorschlag zu haben

*Hierfür werden immer die Feldbenennung und bestimmte Codierungen VOR den Inhalt - in Klammern de Bezeichnung der entsprechenden Spalte - gesetzt, bzw. erfoderliche Felder komplett neu hinzugefügt.   
Am Anfang jeder Zeile braucht Aleph eine 9-Stellige eindeutige Zahl pro Titel.*   

*Manchmal ließ sich der Inhalt einer Spalte direkt in die Datei schreiben, manchmal musst die Spalte zuvor über apply aufbereitet werden.* 

## 1. Aufbereiten der neuen Titel

In [112]:
df_aleph_einspielen = df_aleph_neu_einspielen           #zur vereinfachten Wiederverwertung des alten Codes

In [113]:
df_aleph_einspielen["020"] = df_aleph_einspielen["object_id"].apply(lambda x: f"020   L $$a{x}") 
del df_aleph_einspielen["object_id"]                                                                      #um das df nicht unnötig anwachsen zu lassen, jeweils alte Spalte löschen

#### Besondere Aufbereitung der Personendaten

*Da bis zu 3 Personen in einer Spalte zu finden sind, werden diese im Discovery nicht getrennt suchbar, darum werden sie gesplittet. Für die Dublettenkontrolle hat sich das als irrelevant erwiesen, darum erfolgt dieser Schritt erst hier.*

In [114]:
person = df_aleph_einspielen["contributor"].str.split(';', expand=True)                         #Für saubere Daten, die Autorenangabe splitten und in getrennte Felder schreiben

df_aleph_einspielen["contributor_1"]= person[0]
df_aleph_einspielen["contributor_2"]= person[1]
df_aleph_einspielen["contributor_3"]= person[2]

df_aleph_einspielen["contributor_1"]= df_aleph_einspielen["contributor_1"].replace(np.nan, '', regex=True)
df_aleph_einspielen["contributor_2"]= df_aleph_einspielen["contributor_2"].replace(np.nan, '', regex=True)
df_aleph_einspielen["contributor_3"]= df_aleph_einspielen["contributor_3"].replace(np.nan, '', regex=True)


#### Besondere Aufbereitung des Erscheinungsdatum und Erscheinungsjahres

*In der Auswahl unserer Titel befinden sich auch im Erscheinen befindliche Titel der kommenden Monate. Diese Information möchten wir gerne im Discovery sichtbar machen. Hierfür bleibt uns nur Aleph-Feld 419c, das dem Erscheinungsjahr vorbehalten ist.   
Wunsch ist es: Wenn des Erscheinungsdatum weiter als 10 Tage weg vom heutigen Datum ist, soll das komplette Datum angezeigt werden, ansonsten nur das Erscheinungsjahr.*

Zur Umsetzung muss die Spalte "publication_date" in ein Datum verwandelt werden und nach den genannten Kriterien unterschiedlich angezeigt

In [115]:
today = int(time.strftime('%Y%m%d'))
df_aleph_einspielen["coming_soon"] = np.where(df_aleph_einspielen["publication_date"].astype(int) > today+10, df_aleph_einspielen["publication_date"], np.nan) #zieht die über 10 Tage raus, brauchen nan für Umwandlung in Datum

In [116]:
df_aleph_einspielen["publication_date_soon"] = df_aleph_einspielen["coming_soon"].astype(str).str.replace('00','01')
#df_aleph_einspielen["publication_date_soon"]

In [117]:
df_aleph_einspielen["year"] = df_aleph_einspielen["publication_date_soon"].astype(str).str.slice(start=0,stop=4)    #einfaches Zerlegen in die Datumsbestandteile und anschließendes Zusammenfügen
df_aleph_einspielen["month"] = df_aleph_einspielen["publication_date_soon"].astype(str).str.slice(start=4,stop=6)
df_aleph_einspielen["day"] = df_aleph_einspielen["publication_date_soon"].astype(str).str.slice(start=6,stop=8)

In [118]:
df_aleph_einspielen["full_coming_soon"] = df_aleph_einspielen["year"]+'-'+df_aleph_einspielen["month"]+'-'+df_aleph_einspielen["day"]
df_aleph_einspielen["coming_soon"] = df_aleph_einspielen["full_coming_soon"].astype(str).str.replace('nan--','') 

In [119]:
df_aleph_einspielen["coming_soon"] 

25               
71               
108              
141              
163              
          ...    
471              
472              
473              
475    2021-12-30
476              
Name: coming_soon, Length: 267, dtype: object

In [120]:
df_aleph_einspielen["published"] = np.where(df_aleph_einspielen["coming_soon"] == '', df_aleph_einspielen["publication_year"], '') #Auslesen und Kombinieren der Daten
df_aleph_einspielen["date_combined"] = df_aleph_einspielen["published"]+df_aleph_einspielen["coming_soon"]
df_aleph_einspielen['date_combined']

25           2021
71           2021
108          2021
141          2021
163          2021
          ...    
471          2021
472          2021
473          2021
475    2021-12-30
476          2021
Name: date_combined, Length: 267, dtype: object

In [121]:
df_aleph_einspielen["419b"] = df_aleph_einspielen["publisher"].apply(lambda x: f"419   L $$b{x}") 
df_aleph_einspielen["419c"] = df_aleph_einspielen["date_combined"].apply(lambda x: f"$$c{x}")                  

df_aleph_einspielen["419"] = df_aleph_einspielen["419b"]+df_aleph_einspielen["419c"]                           #Für die Korrekte Eingabe brauche ich Verlag und Jahr in einer Spalte

In [122]:
df_aleph_einspielen["403"] = df_aleph_einspielen["edition_text"].apply(lambda x: f"403   L $$a{x}") 
df_aleph_einspielen["433"] = df_aleph_einspielen["pages"].apply(lambda x: f"433   L $$b{x}")
df_aleph_einspielen["451"] = df_aleph_einspielen["series"].apply(lambda x: f"451   L $$b{x}") 
df_aleph_einspielen["520"] = df_aleph_einspielen["thesis"].apply(lambda x: f"520   L $$a{x}") 
df_aleph_einspielen["540"] = df_aleph_einspielen["isbn_ean"].apply(lambda x: f"540   L $$a{x}") 
df_aleph_einspielen["656"] = df_aleph_einspielen["cover"].apply(lambda x: f"656   L $$u{x}") 
df_aleph_einspielen["750"] = df_aleph_einspielen["description"].apply(lambda x: f"750   L $$a{x}") 

#### Für das Durchzählen der Titel braucht es eine neue Spalte

In [123]:
# hier entsteht eine neue Spalte mit Zahlen ab 1 durchgehend gezählt, die für den korrekten Import der Daten in Aleph nötig ist
x = df_aleph_einspielen.shape[0]   
df_aleph_einspielen["id"] = range(1,x+1)                                                       #Notwendig ist die Zählung ab 1, da Aleph sonst nicht korrekt einließt
df_aleph_einspielen["id"] = df_aleph_einspielen["id"].apply(lambda x: f"{x:09d}")              #Die Zahl muss 9-Stellig aufgefüllt werden



#### Vorbereitungen abgeschlossen, jetzt das Schreiben der Datei im Aleph-Sequential-Format:

In [124]:
with open("./output/pda_ges02", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_aleph_einspielen.index:
        fa.write(df_aleph_einspielen["id"][i]+' LDR   L -----nM2.01200024------h'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["020"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 030   L $$aaz||rrrza||||'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 051   L $$am|||||||m|||||||'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 077   L $$aMonographie'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 078   L $$aSchweitzer'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 082   L $$azum Bestellen'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 100   L $$a'+df_aleph_einspielen["contributor_1"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 104   L $$a'+df_aleph_einspielen["contributor_2"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 108   L $$a'+df_aleph_einspielen["contributor_3"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 331   L $$a'+df_aleph_einspielen["title_sep"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 335   L $$a'+df_aleph_einspielen["subtitle_all"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["403"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["419"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["433"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["451"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["520"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["540"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 655   L $$zOrder me$$umailto:bib@mpifg.de?subject=Bestellwunsch'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["656"][i]+'$$3Cover\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["750"][i]+'\n')

<hr>

### 2. Daten für Update

*es wird anhand der Spalte "last_modified" geprüft, ob die Titel seit dem letzten Einspielen ein Update erfahren haben*


In [152]:
tday = datetime.datetime.now()
td = datetime.timedelta(days = 10)
u = tday - td
df_in_aleph_update['timespan'] = u                                                           # Einfügen einer Spalte mit Datum vor 10 Tagen als Basis für Abfrage zu Update-Notwendigkeit
print(u)

2021-11-21 14:19:10.045322


In [161]:
df_in_aleph_update['last_modified'] = pd.to_datetime(df_in_aleph_update.last_modified)          #Umwandlung, da Spalteninhalt object ist
df_in_aleph_update['last_modified']

In [170]:
df_in_aleph_update['update_true'] = np.where(df_in_aleph_update['last_modified'] < df_in_aleph_update['timespan'] , df_in_aleph_update["mod_date"], np.nan) #Abgleich der beiden Datumsspalten zur Entscheidung, ob Update nötig ist
df_in_aleph_update['update_true']

0       20211118
1            NaN
2       20211119
3       20211019
4            NaN
          ...   
5735         NaN
5736    20211119
5737         NaN
5738         NaN
5739         NaN
Name: update_true, Length: 5735, dtype: object

In [171]:
df_in_aleph_update_true = df_in_aleph_update[df_in_aleph_update['update_true'].notna()]  # damit habe ich alle Titel, die tatsächlich verändert wurden
#df_in_aleph_update_true.info()

#### Erstellen der Update-Export-Datei

In [173]:
df_in_aleph_update_true["020"] = df_in_aleph_update_true["object_id"].apply(lambda x: f"020   L $$a{x}") 
del df_in_aleph_update_true["object_id"]                                                                      #um das df nicht unnötig anwachsen zu lassen, jeweils alte Spalte löschen

In [174]:
#Aufbereitung Personendaten

person = df_in_aleph_update_true["contributor"].str.split(';', expand=True)                         #Für saubere Daten, die Autorenangabe splitten und in getrennte Felder schreiben

df_in_aleph_update_true["contributor_1"]= person[0]
df_in_aleph_update_true["contributor_2"]= person[1]
df_in_aleph_update_true["contributor_3"]= person[2]

df_in_aleph_update_true["contributor_1"]= df_in_aleph_update_true["contributor_1"].replace(np.nan, '', regex=True)
df_in_aleph_update_true["contributor_2"]= df_in_aleph_update_true["contributor_2"].replace(np.nan, '', regex=True)
df_in_aleph_update_true["contributor_3"]= df_in_aleph_update_true["contributor_3"].replace(np.nan, '', regex=True)


In [175]:
#Aufbereitung Erscheinungsdatum 

df_in_aleph_update_true["coming_soon"] = np.where(df_in_aleph_update_true["publication_date"].astype(int) > today+10, df_in_aleph_update_true["publication_date"], np.nan)

df_in_aleph_update_true["publication_date_soon"] = df_in_aleph_update_true["coming_soon"].astype(str).str.replace('00','01')

df_in_aleph_update_true["year"] = df_in_aleph_update_true["publication_date_soon"].astype(str).str.slice(start=0,stop=4)    #einfaches Zerlegen in die Datumsbestandteile und anschließendes Zusammenfügen
df_in_aleph_update_true["month"] = df_in_aleph_update_true["publication_date_soon"].astype(str).str.slice(start=4,stop=6)
df_in_aleph_update_true["day"] = df_in_aleph_update_true["publication_date_soon"].astype(str).str.slice(start=6,stop=8)

df_in_aleph_update_true["full_coming_soon"] = df_in_aleph_update_true["year"]+'-'+df_in_aleph_update_true["month"]+'-'+df_in_aleph_update_true["day"]
df_in_aleph_update_true["coming_soon"] = df_in_aleph_update_true["full_coming_soon"].astype(str).str.replace('nan--','') 

df_in_aleph_update_true["published"] = np.where(df_in_aleph_update_true["coming_soon"] == '', df_in_aleph_update_true["publication_year"], '') #Auslesen und Kombinieren der Daten
df_in_aleph_update_true["date_combined"] = df_in_aleph_update_true["published"]+df_in_aleph_update_true["coming_soon"]
df_in_aleph_update_true['date_combined']

0             2021
2             2021
3             2021
5             2021
7             2021
           ...    
5729    2021-12-20
5731          2021
5733          2021
5734          2021
5736          2021
Name: date_combined, Length: 2384, dtype: object

In [176]:
# Bearbeitung der Felder

df_in_aleph_update_true["419b"] = df_in_aleph_update_true["publisher"].apply(lambda x: f"419   L $$b{x}") 
df_in_aleph_update_true["419c"] = df_in_aleph_update_true["date_combined"].apply(lambda x: f"$$c{x}")                  

df_in_aleph_update_true["419"] = df_in_aleph_update_true["419b"]+df_in_aleph_update_true["419c"]                           #Für die Korrekte Eingabe brauche ich Verlag und Jahr in einer Spalte

df_in_aleph_update_true["403"] = df_in_aleph_update_true["edition_text"].apply(lambda x: f"403   L $$a{x}") 
df_in_aleph_update_true["433"] = df_in_aleph_update_true["pages"].apply(lambda x: f"433   L $$b{x}")
df_in_aleph_update_true["451"] = df_in_aleph_update_true["series"].apply(lambda x: f"451   L $$b{x}") 
df_in_aleph_update_true["520"] = df_in_aleph_update_true["thesis"].apply(lambda x: f"520   L $$a{x}") 
df_in_aleph_update_true["540"] = df_in_aleph_update_true["isbn_ean"].apply(lambda x: f"540   L $$a{x}") 
df_in_aleph_update_true["656"] = df_in_aleph_update_true["cover"].apply(lambda x: f"656   L $$u{x}") 
df_in_aleph_update_true["750"] = df_in_aleph_update_true["description"].apply(lambda x: f"750   L $$a{x}") 

In [184]:
# Schreiben der Ausgabedatei, hier kleine Unterschiede zu den neuen Titeln. Vorhandene ids und bestimmte Felder können nicht verändert sein, brauchen also nicht übernommen zu werden.

with open("./output/pda_update", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_in_aleph_update_true.index:
        fa.write(df_in_aleph_update_true["ids"][i]+' 100   L $$a'+df_in_aleph_update_true["contributor_1"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 104   L $$a'+df_in_aleph_update_true["contributor_2"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 108   L $$a'+df_in_aleph_update_true["contributor_3"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 331   L $$a'+df_in_aleph_update_true["title_sep"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 335   L $$a'+df_in_aleph_update_true["subtitle_all"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["403"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["419"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["433"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["451"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["520"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["540"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["656"][i]+'$$3Cover\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["750"][i]+'\n')

<hr>

## Abschließende Dinge

In [36]:
#Abschließender Eintrag in Log-Datei
endtime = time.strftime('%H:%M')

with open ('./log/pda_import_log.txt', 'a') as log:                                                  # Da diese Log-Datei nicht unmittelbar gebraucht wird, hier fortlaufendes Schreiben in eine Datei
    log.write('\n                                     beendet ')
    log.write(endtime)
    log.write("\n============================================================\n\n")

Kopien bestimmter Daten zur Einsicht bzw. für Prüfzwecke

In [37]:
df_aleph_einspielen.to_csv('./output/Eingespielte_Titel_'+date+'.csv') 
df_in_aleph_update_true.to_csv('./output/Update_Titel_'+date+'.csv')

df_in_aleph_nicht_einspielen.to_csv('./output/Aleph_loeschen_ebx_vorh_'+date+'.csv')

<hr>

Folgende Schritte müssen mit den Dateien ausgeführt werden: 


1. Einspielen der Datei pda_ges01 als neue Titel in Aleph, hier dann auch Export der urls und Anreicherung mit der Aleph-ID mittels "mailto_link_skript.ipynb"
2. Einspielen der DAtei pda_update als "Änderungen bestehender Datensätze in Aleph"
3. Einspielen und löschen der Titel die in ges02_loeschen_1 und ges02_loeschen_2 vorhanden sind